In [ ]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# プログラミングのない世界 (1)

## 計算結果を記憶する

* [Cache (computing) - Wikipedia](https://en.wikipedia.org/wiki/Cache_%28computing%29)

### あらかじめすべての可能性について計算して保持する

In [152]:
import itertools
list(itertools.product([0,1], repeat=2))

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [171]:
len(list(itertools.product([0,1], repeat=2)))

4

$2 \times 2 = 4$ の組み合わせについて、``divmod()`` の値 (剰と余の$2$要素) を列挙する:

In [23]:
binary_add = [[divmod(i+j,2) for j in [0,1]] for i in [0,1]]
binary_add

[[(0, 0), (0, 1)], [(0, 1), (1, 0)]]

In [160]:
len(binary_add)

2

In [153]:
binary_add[0]

[(0, 0), (0, 1)]

In [154]:
binary_add[1]

[(0, 1), (1, 0)]

In [161]:
[len(l) for l in binary_add]

[2, 2]

In [155]:
binary_add[0][0]

(0, 0)

In [156]:
binary_add[0][1]

(0, 1)

In [157]:
binary_add[1][0]

(0, 1)

In [158]:
binary_add[1][1]

(1, 0)

In [13]:
[binary_add[i][j] for i,j in itertools.product([0,1], repeat=2)]

[(0, 0), (0, 1), (0, 1), (1, 0)]

参考) Pythonのリストは可変長なのでサイズ (型) を取れないが、Numpy行列に変換すると $2 \times 2 \times 2$ であることが判る:
* リストと同じようにアクセスできるが、添字を連続して表記することもできる

In [178]:
import numpy as np
x = np.array(binary_add)
x.shape

(2, 2, 2)

In [180]:
x[0]

array([[0, 0],
       [0, 1]])

In [181]:
x[0][0]

array([0, 0])

In [183]:
x[0,0]

array([0, 0])

参考) Pythonのディクショナリを使う:
* 要素のアクセス方法はリストと同じ

In [22]:
binary_add = {i : {j : divmod(i+j,2) for j in [0,1]} for i in [0,1]}
binary_add

{0: {0: (0, 0), 1: (0, 1)}, 1: {0: (0, 1), 1: (1, 0)}}

In [20]:
[binary_add[i][j] for i,j in itertools.product([0,1], repeat=2)]

[(0, 0), (0, 1), (0, 1), (1, 0)]

### 計算するごとに結果をキャッシュに保持する

計算結果をキャッシュに記憶して、次の計算時はキャッシュされた値を返す:
* キャッシュが値を持っていない状態を表現する
  - ２行２列の整数型のデータ行列を作る (``divmod()`` の戻り値が2要素なので、$2 \times 2 \times 2$ 行列になる)
    - ``np.empty((2,2,2), dtype=int)``
  - 同じサイズの行列を作って、データにキャッシュされた値を保持しているか (`False`) 否か (`True`) を設定できるようにする
    - ``np.ma.array(np.empty((2,2,2), dtype=int), mask=np.ones((2,2,2)), dtype=int))``
* データの特別な値をキャッシュに保持されているか否かの判定に使うこともできる
  - 整数: ``0`` (``False``と等価。今回は、計算結果が``0``になることがあるので使えない)
  - 整数: ``-1`` (計算結果が負の値になる時は使えない)
  - 浮動小数: ``NaN``

In [ ]:
import numpy as np

空の行列 (値は初期化されていない):

In [100]:
np.empty((2,2,2), dtype=int)

array([[[4523236272, 4523236272],
        [4523236272, 4523236272]],

       [[4523236272, 4523236272],
        [4457646384, 4457646352]]])

全ての要素が $1$ (``True``) の行列:

In [106]:
np.ones((2,2,2), dtype=int)

array([[[1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1]]])

組み合わせてマスク付き行列を作る:

In [139]:
x = np.ma.array(np.empty((2,2,2), dtype=int), mask=np.ones((2,2,2), dtype=int))
x

masked_array(
  data=[[[--, --],
         [--, --]],

        [[--, --],
         [--, --]]],
  mask=[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],
  fill_value=999999,
  dtype=int64)

In [140]:
x[0,0]

masked_array(data=[--, --],
             mask=[ True,  True],
       fill_value=999999,
            dtype=int64)

値を代入していないので `masked` 状態である:

In [141]:
x[0,0][0], x[0,0][1]

(masked, masked)

In [142]:
x[0,0][0] is np.ma.masked, x[0,0][1] is np.ma.masked

(True, True)

In [143]:
x[0,0] = (0, 1)
x

masked_array(
  data=[[[0, 1],
         [--, --]],

        [[--, --],
         [--, --]]],
  mask=[[[False, False],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],
  fill_value=999999)

In [144]:
x[0,0][0], x[0,0][1]

(0, 1)

In [115]:
x[0,0][0] is np.ma.masked, x[0,0][1] is np.ma.masked

(False, False)

In [129]:
class binary_op:

    def __init__(self):
        self.cached_add = np.ma.array(np.empty((2,2,2), dtype=int), mask=np.ones((2,2,2), dtype=int))
        
    def add(self,a,b):
        if a in [0,1] and b in [0,1]:
            if self.cached_add[a,b,0] is np.ma.masked or self.cached_add[a,b,1] is np.ma.masked:
                self.cached_add[a,b] = divmod(a+b,2)
            return tuple(self.cached_add.data[a,b])
        else:
            raise ValueError

In [130]:
bo = binary_op()

In [131]:
bo.cached_add

masked_array(
  data=[[[--, --],
         [--, --]],

        [[--, --],
         [--, --]]],
  mask=[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],
  fill_value=999999,
  dtype=int64)

一度計算すると結果がキャッシュに保持される:

In [132]:
bo.add(1,1), divmod(1+1,2)

((1, 0), (1, 0))

In [133]:
bo.cached_add

masked_array(
  data=[[[--, --],
         [--, --]],

        [[--, --],
         [1, 0]]],
  mask=[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [False, False]]],
  fill_value=999999)

二度目からはキャッシュされた値が戻る:

In [134]:
bo.add(1,1)

(1, 0)